# Setup

In [2]:
import numpy as np
import polars as pl
import pandas as pd
from joblib import dump, load
from pathlib import Path
from lisa.trace import Trace

from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

pl.enable_string_cache()

In [65]:
TRAINING_PATH = Path("../training")
TESTING_PATH = Path("../testing")
MODEL_PATH = Path("../models")

ANA = 'analysis'

GB6 = TRAINING_PATH / 'geekbench_idle_trace_rq_3_1112'
GB6_2 = TRAINING_PATH / 'geekbench_idle_trace_rq_3_1312'
JET = TRAINING_PATH / 'jetnews_idle_trace_rq_10_1112'
JET_2 = TRAINING_PATH / 'jetnews_idle_trace_rq_10_1212'
SPE = TRAINING_PATH / 'speedometer_idle_trace_rq_10_1112'

GB6_T = TESTING_PATH / 'geekbench_idle_trace_rq_test_3_1312'
JET_T = TESTING_PATH / 'jetnews_idle_trace_rq_test_10_1412'
SPE_T = TESTING_PATH / 'speedometer_idle_trace_rq_test_10_1312'
DRA_T = TESTING_PATH / 'drarm_idle_trace_rq_10_1212'

GB6_M = TESTING_PATH / 'geekbench_menu_idle_3_1412'
JET_M = TESTING_PATH / 'jetnews_menu_idle_10_1412'
SPE_M = TESTING_PATH / 'speedometer_menu_idle_10_1412'
DRA_M = TESTING_PATH / 'drarm_menu_idle_10_1512'

WORKLOADS = [GB6, JET, SPE]

# CPUIdle baseline accuracy

In [68]:
def workload_idle_baseline_accuracy(path):
    print('Baseline for', path)
    cpu_idle = pl.read_parquet(path / ANA / 'cpu_idle.pqt')
    wakeups = cpu_idle.filter(pl.col('state') == -1)
    cpu_idle_miss = pl.read_parquet(path / ANA / 'cpu_idle_miss.pqt')
    
    print('accuracy', 1 - cpu_idle_miss.height / wakeups.height)
    print('misses', cpu_idle_miss.height / wakeups.height)
    print('too deep', cpu_idle_miss.filter(pl.col('state') == 1).height / wakeups.height, 'too shallow', cpu_idle_miss.filter(pl.col('state') == 0).height / wakeups.height)
    print()

    return (wakeups.height, cpu_idle_miss.height)

print('*** Menu idle accuracy ***\n')
wakeups_gb6_m, misses_gb6_m = workload_idle_baseline_accuracy(GB6_M)
wakeups_jet_m, misses_jet_m = workload_idle_baseline_accuracy(JET_M)
wakeups_spe_m, misses_spe_m = workload_idle_baseline_accuracy(SPE_M)
wakeups_dra_m, misses_dra_m = workload_idle_baseline_accuracy(DRA_M)

wakeups_total_m = wakeups_gb6_m + wakeups_jet_m + wakeups_spe_m + wakeups_dra_m
misses_total_m = misses_gb6_m + misses_jet_m + misses_spe_m + misses_dra_m

print('Total menu baseline')
print('accuracy', 1 - misses_total_m / wakeups_total_m)
print('misses',  misses_total_m / wakeups_total_m)
print()

print('*** TEO idle accuracy ***')

wakeups_gb6, misses_gb6 = workload_idle_baseline_accuracy(GB6)
wakeups_gb6_2, misses_gb6_2 = workload_idle_baseline_accuracy(GB6_2)
wakeups_jet, misses_jet = workload_idle_baseline_accuracy(JET)
wakeups_jet_2, misses_jet_2 = workload_idle_baseline_accuracy(JET_2)
wakeups_spe, misses_spe = workload_idle_baseline_accuracy(SPE)
wakeups_dra, misses_dra = workload_idle_baseline_accuracy(DRA_T)

wakeups_total = wakeups_gb6 + wakeups_gb6_2 + wakeups_jet + wakeups_jet_2 + wakeups_spe + wakeups_dra
misses_total = misses_gb6 + misses_gb6_2 + misses_jet + misses_jet_2 + misses_spe + misses_dra

print('Total TEO baseline')
print('accuracy', 1 - misses_total / wakeups_total)
print('misses',  misses_total / wakeups_total)

*** Menu idle accuracy ***

Baseline for ../testing/geekbench_menu_idle_3_1412
accuracy 0.816481372881879
misses 0.18351862711812092
too deep 0.1578486990339208 too shallow 0.02566992808420013

Baseline for ../testing/jetnews_menu_idle_10_1412
accuracy 0.6962507146510999
misses 0.30374928534890017
too deep 0.2697473562088553 too shallow 0.03400192914004487

Baseline for ../testing/speedometer_menu_idle_10_1412
accuracy 0.7800750738896148
misses 0.21992492611038514
too deep 0.18292211339068337 too shallow 0.037002812719701766

Baseline for ../testing/drarm_menu_idle_10_1512
accuracy 0.8691868299015306
misses 0.1308131700984694
too deep 0.11541427240054901 too shallow 0.015398897697920407

Total menu baseline
accuracy 0.8250149988085469
misses 0.17498500119145316

*** TEO idle accuracy ***
Baseline for ../training/geekbench_idle_trace_rq_3_1112
accuracy 0.7830149330655424
misses 0.21698506693445763
too deep 0.08570174250092319 too shallow 0.13128332443353444

Baseline for ../training/gee

# Training data

In [5]:
SLEEP_COUNT = 3_000_003

idle_data_gb6 = pl.concat([
    pl.read_parquet(GB6 / ANA / 'idle_decisions.pqt'),
    pl.read_parquet(GB6_2 / ANA / 'idle_decisions.pqt')
])
print('gb6', idle_data_gb6.shape)

idle_data_jet = pl.concat([
  pl.read_parquet(JET / ANA / 'idle_decisions.pqt'),
  pl.read_parquet(JET_2 / ANA / 'idle_decisions.pqt')  
])
print('jet', idle_data_jet.shape)

idle_data_spe = pl.read_parquet(SPE / ANA / 'idle_decisions.pqt')
print('spe', idle_data_spe.shape)

idle_data_dra = pl.read_parquet(DRA / ANA / 'idle_decisions.pqt')
print('dra', idle_data_dra.shape)

idle_data = pl.concat([idle_data_gb6[:SLEEP_COUNT], idle_data_jet[:SLEEP_COUNT], idle_data_spe[:SLEEP_COUNT]])
print('total', idle_data.shape)
idle_data.head()

gb6 (3292802, 28)
jet (5234560, 28)
spe (3987020, 28)
dra (13051471, 28)
total (9000009, 28)


sleep_id,cpu,sleep_length,time_span,measured,util,max_cap,s0hit,s0int,s0rec,s1hit,s1int,s1rec,nr_running,cfs_nr_running,nr_switches,rq_cpu_time,sched_count,ttwu_count,yld_count,timer_state,duration_state,state,hit,goal_state,iteration,kernel,wa_path
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32,i32,str,str
825123,0,1336467,1367147,-1,1,160,2175,1389,1,4641,0,0,0,0,1988727,292746426980,2071140,983841,60405,0,0,0,1,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825124,0,6561466,566366,408641,1,160,2928,1216,1,4061,0,0,0,0,1988727,292746426980,2071140,983842,60405,1,0,1,0,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825125,0,5876082,393636,230825,1,160,2562,2088,2,3554,0,0,0,0,1988731,292746530414,2071144,983846,60405,1,0,1,0,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825126,0,5396631,573079,422313,1,160,2242,2851,3,3110,0,0,0,0,1988734,292746603045,2071147,983849,60405,1,0,1,0,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825127,0,4771387,4771387,-1,1,160,1962,3519,4,2722,0,0,0,0,1988736,292746651222,2071149,983851,60405,1,1,0,1,1,1,"""6.3.0-mainline…","""geekbench_idle…"


In [6]:
idle_data_dra_processed = idle_data_dra.with_columns(
    pl.col('nr_switches').diff().over('cpu'), pl.col('rq_cpu_time').diff().over('cpu'), pl.col('sched_count').diff().over('cpu'), pl.col('ttwu_count').diff().over('cpu'), pl.col('yld_count').diff().over('cpu')
).drop_nulls()

idle_data_processed = idle_data.with_columns(
    pl.col('nr_switches').diff().over('cpu'), pl.col('rq_cpu_time').diff().over('cpu'), pl.col('sched_count').diff().over('cpu'), pl.col('ttwu_count').diff().over('cpu'), pl.col('yld_count').diff().over('cpu')
).drop_nulls()
idle_data_processed.head()

sleep_id,cpu,sleep_length,time_span,measured,util,max_cap,s0hit,s0int,s0rec,s1hit,s1int,s1rec,nr_running,cfs_nr_running,nr_switches,rq_cpu_time,sched_count,ttwu_count,yld_count,timer_state,duration_state,state,hit,goal_state,iteration,kernel,wa_path
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32,i32,str,str
825124,0,6561466,566366,408641,1,160,2928,1216,1,4061,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825125,0,5876082,393636,230825,1,160,2562,2088,2,3554,0,0,0,0,4,103434,4,4,0,1,0,1,0,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825126,0,5396631,573079,422313,1,160,2242,2851,3,3110,0,0,0,0,3,72631,3,3,0,1,0,1,0,0,1,"""6.3.0-mainline…","""geekbench_idle…"
825127,0,4771387,4771387,-1,1,160,1962,3519,4,2722,0,0,0,0,2,48177,2,2,0,1,1,0,1,1,1,"""6.3.0-mainline…","""geekbench_idle…"
825128,0,3942733,4092855,-1,1,160,1717,3080,4,3406,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,"""6.3.0-mainline…","""geekbench_idle…"


In [10]:
def training_strip(df):
    df = df.drop([
        'sleep_id', 'time_span', 'measured', 'state', 'hit', 'timer_state', 'duration_state', 'cpu', 'iteration', 'kernel', 'wa_path',
    ]).to_pandas()
    X = df.drop(['goal_state'], axis=1)
    y = df['goal_state']
    return (X, y)

print('TEO decision accuracy', 1 - idle_data_processed.filter(pl.col('state') != pl.col('goal_state')).height / idle_data_processed.height)
X, y = training_strip(idle_data_processed)
print(X.shape, y.shape)
display(X.head())
y.head()

TEO decision accuracy 0.7948968005670222
(9000001, 16) (9000001,)


,sleep_length,util,max_cap,s0hit,s0int,s0rec,s1hit,s1int,s1rec,nr_running,cfs_nr_running,nr_switches,rq_cpu_time,sched_count,ttwu_count,yld_count
0,6561466,1,160,2928,1216,1,4061,0,0,0,0,0,0,0,1,0
1,5876082,1,160,2562,2088,2,3554,0,0,0,0,4,103434,4,4,0
2,5396631,1,160,2242,2851,3,3110,0,0,0,0,3,72631,3,3,0
3,4771387,1,160,1962,3519,4,2722,0,0,0,0,2,48177,2,2,0
4,3942733,1,160,1717,3080,4,3406,0,0,0,0,0,0,0,0,0


0    0
1    0
2    0
3    1
4    1
Name: goal_state, dtype: int32

# RandomForest

## Train-test split

In [ ]:
# overfitting at max_depth > 16-17
# optimal accuracy around max_depth == 15

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

base_forest = RandomForestClassifier(n_estimators=10, n_jobs=8, max_depth=16)
base_forest.fit(X_train, y_train)
print("RandomForest accuracy on full set: {:.3f}".format(base_forest.score(X, y)))
print("RandomForest accuracy on training set: {:.3f}".format(base_forest.score(X_train, y_train)))
print("RandomForest accuracy on test set: {:.3f}".format(base_forest.score(X_test, y_test)))

RandomForest accuracy on full set: 0.817
RandomForest accuracy on training set: 0.818
RandomForest accuracy on test set: 0.813


## Grid Search CV

In [25]:
parameters = {'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'criterion': ('gini', 'entropy', 'log_loss')}
clf = GridSearchCV(RandomForestClassifier(n_jobs=8), parameters)
clf.fit(X_train, y_train)
print(clf.cv_results_['mean_test_score'].mean(), clf.cv_results_['mean_test_score'])
print('Best RandomForest accuracy on full dataset', clf.best_estimator_.score(X, y))
print('Best RandomForest accuracy on test dataset', clf.best_estimator_.score(X_test, y_test))

0.8114302973166003 [0.80672444 0.8071843  0.80714904 0.80721304 0.80740696 0.80717881
 0.80710326 0.80736963 0.80722504 0.80750726 0.80721719 0.80816385
 0.80827111 0.808404   0.80823956 0.80827615 0.80831319 0.80804948
 0.80841985 0.80831393 0.80843807 0.80844474 0.80898133 0.80906385
 0.80940104 0.80917259 0.80920711 0.80927467 0.80941911 0.80955407
 0.80920622 0.809572   0.80930444 0.81020667 0.81040504 0.81032489
 0.81049911 0.81040815 0.8103957  0.81043289 0.81053319 0.81081733
 0.81072548 0.81079215 0.81123467 0.81110815 0.81141259 0.81166267
 0.81169481 0.8118677  0.81196    0.81171541 0.81176281 0.81188904
 0.81199111 0.8119517  0.81220163 0.81230815 0.81230696 0.81234504
 0.81241689 0.81264948 0.81258948 0.81280267 0.8125883  0.81285911
 0.81211689 0.81273037 0.81293911 0.81338296 0.8133723  0.81337837
 0.81359852 0.81332311 0.81359304 0.81358296 0.81389452 0.81288904
 0.81314622 0.81342207 0.81376044 0.81410741 0.81393615 0.81421778
 0.81406711 0.81431837 0.81416519 0.8144684

In [54]:
# print(clf.cv_results_['mean_test_score'].mean(), clf.cv_results_['mean_test_score'])

# forest = clf.best_estimator_
print('RandomForest accuracy on full dataset: ', forest.score(X, y))
print('RandomForest accuracy on test dataset: ', forest.score(X_test, y_test))

RandomForest accuracy on full dataset:  0.8179599091155657
RandomForest accuracy on test dataset:  0.817932969807569


In [50]:
# dump(base_forest, MODEL_PATH / 'rq_gb6_jet_spe_3m_forest_10e_16d_814.joblib')

# dump(forest, MODEL_PATH / 'rq_gb6_jet_spe_3m_log_loss_forest_15e_20d_815.joblib')
forest = load(Path(MODEL_PATH) / 'rq_gb6_jet_spe_3m_forest_10e_16d_814.joblib')

# dump(clf.best_estimator_, Path(MODEL_PATH) / 'gb6_3_forest_80.joblib')
# forest2 = load(Path(MODEL_PATH) / 'gb6_3_forest_80.joblib')

# dump(clf, Path(MODEL_PATH) / 'gb6_3_gridcv_80.joblib')
# clf2 = load(Path(MODEL_PATH) / 'gb6_3_gridcv_80.joblib')

# forest = load(Path(MODEL_PATH) / 'gb6_jet_spe_forest_80.joblib')
# forest.score(X, y)

In [55]:
forest

RandomForestClassifier(max_depth=16, n_estimators=10, n_jobs=8)

In [ ]:
import emlearn
cmodel = emlearn.convert(forest, method='inline', dtype='int32')
# cmodel.save(file='rq_gb6_jet_spe_forest_5e_16d_815.h', name='forest_idle_rq')
cmodel

# Validation

## Load validation data

In [72]:
test_data_gb6 = pl.read_parquet(GB6_T / ANA / 'idle_decisions.pqt')
print('gb6', test_data_gb6.shape)

test_data_jet = pl.read_parquet(JET_T / ANA / 'idle_decisions.pqt')
print('jet', test_data_jet.shape)

test_data_spe = pl.read_parquet(SPE_T / ANA / 'idle_decisions.pqt')
print('spe', test_data_spe.shape)

test_data_dra = pl.read_parquet(DRA_T / ANA / 'idle_decisions.pqt')
print('dra', test_data_dra.shape)

gb6 (1546354, 28)
jet (2603689, 28)
spe (3731326, 28)
dra (13051391, 28)


## GB6 validation

In [62]:
gb6_test_hits = test_data_gb6.filter(pl.col('state') == pl.col('goal_state')).height / test_data_gb6.height
gb6_test_too_deep = test_data_gb6.filter(pl.col('state') > pl.col('goal_state')).height / test_data_gb6.height
gb6_test_too_shallow = test_data_gb6.filter(pl.col('state') < pl.col('goal_state')).height / test_data_gb6.height
print('TEO GB6 test set decision accuracy', gb6_test_hits, gb6_test_too_deep, gb6_test_too_shallow)

X_test_gb6, y_test_gb6 = training_strip(test_data_gb6)
print('RandomForests GB6 test set decision accuracy', forest.score(X_test_gb6, y_test_gb6))

TEO GB6 test set decision accuracy 0.7754104170196475 0.07653098837652957 0.14805859460382292
RandomForests GB6 test set decision accuracy 0.8178715869716766


## Speedometer validation

In [63]:
spe_test_hits = test_data_spe.filter(pl.col('state') == pl.col('goal_state')).height / test_data_spe.height
spe_test_too_deep = test_data_spe.filter(pl.col('state') > pl.col('goal_state')).height / test_data_spe.height
spe_test_too_shallow = test_data_spe.filter(pl.col('state') < pl.col('goal_state')).height / test_data_spe.height
print('TEO Speedometer test set decision accuracy', spe_test_hits, spe_test_too_deep, spe_test_too_shallow)

X_test_spe, y_test_spe = training_strip(test_data_spe)
print('RandomForests Speedometer test set decision accuracy', forest.score(X_test_spe, y_test_spe))

TEO Speedometer test set decision accuracy 0.8284071131817483 0.010008506359401457 0.1615843804588503
RandomForests Speedometer test set decision accuracy 0.8330218265571007


## JetNews validation

In [64]:
jet_test_hits = test_data_jet.filter(pl.col('state') == pl.col('goal_state')).height / test_data_jet.height
jet_test_too_deep = test_data_jet.filter(pl.col('state') > pl.col('goal_state')).height / test_data_jet.height
jet_test_too_shallow = test_data_jet.filter(pl.col('state') < pl.col('goal_state')).height / test_data_jet.height
print('TEO JetNews test set decision accuracy', jet_test_hits, jet_test_too_deep, jet_test_too_shallow)

X_test_jet, y_test_jet = training_strip(test_data_jet)
print('RandomForests JetNews test set decision accuracy', forest.score(X_test_jet, y_test_jet))

TEO JetNews test set decision accuracy 0.7651109637134081 0.03355354652571793 0.2013354897608739
RandomForests JetNews test set decision accuracy 0.7792086535680721


## DrArm validation

In [73]:
dra_test_hits = test_data_dra.filter(pl.col('state') == pl.col('goal_state')).height / test_data_dra.height
dra_test_too_deep = test_data_dra.filter(pl.col('state') > pl.col('goal_state')).height / test_data_dra.height
dra_test_too_shallow = test_data_dra.filter(pl.col('state') < pl.col('goal_state')).height / test_data_dra.height
print('TEO DrArm test set decision accuracy', dra_test_hits, dra_test_too_deep, dra_test_too_shallow)

X_dra, y_dra = training_strip(test_data_dra)
print('RandomForests DrArm decision accuracy', forest.score(X_dra, y_dra))

TEO DrArm test set decision accuracy 0.9502149617615471 0.0009287898891390197 0.048856248349313874
RandomForests DrArm decision accuracy 0.9495296708220603


# MLP

In [32]:
X_mlp, y_mlp = training_strip(idle_data_processed)
X_mlp_train, X_mlp_test, y_mlp_train, y_mlp_test = train_test_split(X_mlp, y_mlp)

scaler = StandardScaler()  
scaler.fit(X_mlp_train)  
X_mlp_train = scaler.transform(X_mlp_train)  
X_mlp_test = scaler.transform(X_mlp_test)  

mlp = MLPClassifier()
mlp.fit(X_mlp_train, y_mlp_train)
print("MLP accuracy on training set: {:.3f}".format(mlp.score(X_mlp_train, y_mlp_train)))
print("MLP accuracy on test set: {:.3f}".format(mlp.score(X_mlp_test, y_mlp_test)))

MLP accuracy on training set: 0.806
MLP accuracy on test set: 0.806


In [33]:
dump(mlp, MODEL_PATH / 'rq_gb6_jet_spe_3m_mlp_808.joblib')

print('MLP accuracy on test dataset: ', mlp.score(X_mlp_test, y_mlp_test))

MLP accuracy on test dataset:  0.805999641777937
